# Исследование надёжности заёмщиков
**Заказчик** — кредитный отдел банка.

**Цель:**
  Необходимо выяснить, влияет ли семейное положение и количество детей заёмщика на факт погашения кредита в срок. 
  
**Входные данные от банка** — статистика о платёжеспособности клиентов в виде таблицы csv.

*Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.*

<a id="contents"></a>
## [План исследования](#contents)
- [Описание данных](#data_description)
- [Изучение общей информации](#start)
- [Предобработка данных](#preprocessing)
 * [Обработка пропусков](#null)
 * [Замена типа данных](#replace)
 * [Обработка дубликатов](#duplicates)
 * [Лемматизация](#lemm)
 * [Категоризация данных](#category)
- [Анализ](#analysis)
- [Общий вывод](#general)


<a id="data_description"></a>
## Описание данных
Таблица состоит из следующих столбцов:
- children — количество детей в семье
- days_employed — трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — образование клиента
- education_id — идентификатор образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — доход в месяц

<a id="start"></a>
## Изучение общей информации

In [1]:
#импортируем библиотеки
import pandas as pd 
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
df = pd.read_csv('/datasets/data.csv') #распакуем файл
df.head(15) #изучим верхние строки таблицы


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.tail(15) #изучим последние строки таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


In [3]:
df.info() #получим сведения о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# подсчитаем суммарное количество пропусков
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df.describe() #выведем статистические сведения

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
#рассмотрим уникальные значения данных для каждого столбца и преобразуем для наглядности при помощи to_frame
for row in df: 
    display(df[row].value_counts().to_frame())
    

,children
0,14149
1,4818
2,2055
3,330
20,76
-1,47
4,41
5,9


,days_employed
-986.927316,1
-7026.359174,1
-4236.274243,1
-6620.396473,1
-1238.560080,1
...,...
-2849.351119,1
-5619.328204,1
-448.829898,1
-1687.038672,1


,dob_years
35,617
40,609
41,607
34,603
38,598
42,597
33,581
39,573
31,560
36,555


,education
среднее,13750
высшее,4718
СРЕДНЕЕ,772
Среднее,711
неоконченное высшее,668
ВЫСШЕЕ,274
Высшее,268
начальное,250
Неоконченное высшее,47
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29


,education_id
1,15233
0,5260
2,744
3,282
4,6


...

,gender
F,14236
M,7288
XNA,1


,income_type
сотрудник,11119
компаньон,5085
пенсионер,3856
госслужащий,1459
безработный,2
предприниматель,2
в декрете,1
студент,1


,debt
0,19784
1,1741


,total_income
112874.418757,1
133912.272223,1
182036.676828,1
122421.963500,1
198271.837248,1
...,...
133299.194693,1
115080.782380,1
84896.781597,1
153838.839212,1


,purpose
свадьба,797
на проведение свадьбы,777
сыграть свадьбу,774
операции с недвижимостью,676
покупка коммерческой недвижимости,664
покупка жилья для сдачи,653
операции с жильем,653
операции с коммерческой недвижимостью,651
жилье,647
покупка жилья,647


**Вывод**


- отрицательные значения в колонке days_emploed
- разный регистр в колонке education
- наличие пропусков в колонке days_employed, total_income
- ошибочные значения в колонке children и в колонке gender
- нулевые значения в колонке dob_years
- дубликаты в целевых использованиях в колонке purpose

[Наверх к оглавлению](#contents)

<a id="preprocessing"></a>
##  Предобработка данных

<a id="null"></a>
### Обработка пропусков

In [7]:
#при изучении информации в колонке gender была одна строка с пропущенным значением.Исключаем её из выборки
df = df[df['gender'] != 'XNA']


In [8]:
#найдем предполагаемые причины пропусков в колонках days_employed, total_income. 
#установим, в одинаковых ли строках отсутствуют значения.
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [9]:
#значения трудового стажа пропущены тогда, когда пропущены значения ежемесячного дохода.
#проверим связан ли тип занятости с пропусками в данных
df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [10]:
#все пропуски находятся в разных категориях занятости, значит, будем заполнять пропуски по медианному значению каждого типа
#для начала изучим колонку 'days_employed' более подробно:

count = 0
for num in df['days_employed']:
    if num<0:
        count+=1
    
count 


15905

In [11]:
#отрицательных значений в колонке 'days_employed' ~ 74%. Больше похоже на ошибку при заполнении

'Доля отрицательных значений в колонке c количеством дней стажа: {:.2%}'. format(15905 / 21525)

'Доля отрицательных значений в колонке c количеством дней стажа: 73.89%'

In [12]:
#посчитаем количество значений > 0 по каждому типу занятости
df_grouped = df.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})
df_grouped

days_employed                          
                        count           mean <lambda_0>
income_type                                            
безработный                 2  366413.652744        2.0
в декрете                   1   -3296.759962        0.0
госслужащий              1312   -3399.896902        0.0
компаньон                4576   -2111.470404        0.0
пенсионер                3443  365003.491245     3443.0
предприниматель             1    -520.848083        0.0
сотрудник               10014   -2326.499216        0.0
студент                     1    -578.751554        0.0

In [13]:
#гипотеза в ошибочном заполнении потдвердилась, отриц.значения только у работающего персонала
#преобразуем отриц.значения
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].describe()

count     19350.000000
mean      66918.065141
std      139033.698578
min          24.141633
25%         926.990457
50%        2194.218768
75%        5538.423086
max      401755.400475
Name: days_employed, dtype: float64

In [14]:
#для исправления пропущенных значений воспользуемся медианными значениями по возрасту. 
#выделяем возрастные категории клиентов: в качестве границ категорий выбраны квантили (25%, 50% и 75%).
age_statistics = df['dob_years'].describe()
age_statistics[4:7]

25%    33.0
50%    42.0
75%    53.0
Name: dob_years, dtype: float64

In [15]:
# напишем функцию, которая принимает на вход возраст клиента и возвращает возрастную категорию
def age_group(age):
    if age <= age_statistics[4]: return 'до 34'
    elif age_statistics[4] < age <= age_statistics[5]: return '34-43'
    elif age_statistics[5] < age <= age_statistics[6]: return '43-53'
    else: return '53+'
    
df['age_group'] = df['dob_years'].apply(age_group)
df.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,34-43
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,34-43


In [16]:
#найдем медианное значение трудового стажа для каждой возрастной группы.
medians_of_days_employed = df.groupby('age_group')['days_employed'].median()
round(medians_of_days_employed, 0)

age_group
34-43      1812.0
43-53      2441.0
53+      342859.0
до 34      1147.0
Name: days_employed, dtype: float64

In [17]:
# заменим пропущенные значения трудового стажа на медиану из возрастных категорий
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '34-43'), 'days_employed'] = medians_of_days_employed[0]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '43-53'), 'days_employed'] = medians_of_days_employed[1]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '53+'), 'days_employed'] = medians_of_days_employed[2]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == 'до 34'), 'days_employed'] = medians_of_days_employed[3]


In [18]:
#найдем медианные значения возрастов клиентов для каждого типа занятости:
age_medians = df.groupby('income_type')['dob_years'].median()
age_medians

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [19]:
#заменим нулевые значения на медианы по каждому типу занятости.
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_medians[6]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_medians[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_medians[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_medians[2]

# Проверим, все ли нули мы исправили.
df.loc[df['dob_years'] == 0, 'dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

In [20]:
#заменим пустые значения в ежемесячном доходе на медианные по каждому типу занятости.
#найдем медианные уровни дохода для каждого типа занятости
medians_total_income = df.groupby('income_type')['total_income'].median()
round(medians_total_income)

income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172319.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64

In [21]:
# Заменим пропущенные значения ежемесячного дохода на медианы по типам занятости.
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
age_group           21524 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.9+ MB


In [23]:
#предположим, что ошибочные значения в колонке children (20)=2 и (-1)=1 и заменим их на новые 
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
df['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [24]:
#приведем все значения к одному регистру
df['education'] = df['education'].str.lower()
# в столбце education теперь 5 уникальных значений
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

- положительные значения в колонке days_emploed;
- регистр в колонке education единый;
- удалено ошибочное значение в колонке gender;
- обработаны пропущенные значения по ежемесячному доходу на основе медиан для каждого типа занятости;
- исправлены значения нулевого возраста на медианы соответствующего типа занятости;
- обработаны пропущенные значения в данных по трудовому стажу на основе медиан для каждой возрастной категории;
- заменены ошибочные значения в колонке с количеством детей.

[Наверх к оглавлению](#contents)

<a id="replace"></a>
### Замена типа данных

In [25]:
# изменим тип значений с вещественного на целый  для более удобного формата считывания информации c помощью функции'astype()',
#т.к. пропущенные значения уже обработаны
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.dtypes

children              int64
days_employed         int64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
age_group            object
dtype: object

**Вывод**

Таблица имеет отформатированный вид для дальнейшей работы.

[Наверх к оглавлению](#contents)

<a id="duplicates"></a>
### Обработка дубликатов

In [26]:
#посчитаем количество дубликатов 
df.duplicated().sum()

71

In [27]:
#найдем число идентичных строк
df.duplicated(keep = False).sum()

137

In [28]:
#посмотрим как это выглядит в таблице
df[df.duplicated(keep = False)].sort_values('dob_years', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
7938,0,342859,71.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
9604,0,342859,71.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
6537,0,342859,71.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
5865,0,342859,66.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+
9528,0,342859,66.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+


In [29]:
#дубликаты похожи на техническую ошибку либо ошибку при вводе данных
#удалим идентичных строки
df = df.drop_duplicates().reset_index(drop = True)

In [30]:
#проверим еще раз на наличие дубликатов
df.duplicated().sum()

0

**Вывод**

Необходимо в дальнейшем, кто будет использовать таблицу для работы, выяснить, влияет ли человеческий фактор на дубликаты или это техническая ошибка.

[Наверх к оглавлению](#contents)

<a id="lemm"></a>
### Лемматизация

In [31]:
#рассмотрим все варианты целей для кредитования
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [32]:
#Посчитаем, сколько раз упоминаются различные цели с помощью контейнера Counter из модуля collections
#используем функцию 'join()', чтобы преобразовать перечисляемую последовательность в список
lemmas = m.lemmatize('; '.join(df['purpose']))
Counter(lemmas)

Counter({'покупка': 5896,
         ' ': 33569,
         'жилье': 4460,
         '; ': 21452,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6350,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

In [33]:
#среди целей выделяется 6 типов. Выведем их вручную
goal = ["строительство", "образование", "свадьба", "недвижимость", "автомобиль", "жилье"]

# проведем лемматизацию,
# одновременно заменив полученный список лемм в каждой строке на главное ключевое слово из списка категорий

def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in goal:
        if word in lemma:
            lemma = word
    return lemma
#создадим новый столбец (purpose_group) с категориями на основании результатов лемматизации
df['purpose_group'] = df['purpose'].apply(lemmatize)        
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,жилье
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,жилье
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 34,жилье
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,43-53,жилье
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,43-53,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,34-43,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,34-43,жилье


In [34]:
#посчитаем все возможные категории целей кредита
df['purpose_group'].value_counts()

недвижимость     4472
жилье            4460
автомобиль       4306
образование      4013
свадьба          2324
строительство    1878
Name: purpose_group, dtype: int64

In [35]:
#объединим две категории 'жилье' и 'нежвижимость' в одну
df.loc[df['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
df['purpose_group'].value_counts()

недвижимость     8932
автомобиль       4306
образование      4013
свадьба          2324
строительство    1878
Name: purpose_group, dtype: int64

**Вывод**

Выделили 5 основных категорий целевых кредитов. 

[Наверх к оглавлению](#contents)

<a id="category"></a>
### Категоризация данных

*Ранее уже были созданы столбец с категориями по целям кредита 'purpose_category' и столбец с группированием по возрасту 'age_group'. Создадим еще одну категоризацию - по доходам. Для этого воспользуемся квантилями 25%, 50% и 75%:*

In [36]:
total_statistics = df['total_income'].describe()
total_statistics[4:7]

25%    107620.0
50%    142594.0
75%    195818.0
Name: total_income, dtype: float64

In [37]:
# создадим функцию:
def specify_income_group(income):
    if income <= total_statistics[4]: return 'до 107620'
    elif total_statistics[4] < income <= total_statistics[5]: return 'от 107620 до 142594'
    elif total_statistics[5] < income <= total_statistics[6]: return 'от 142594 до 195818'
    else: return 'выше 195818'

In [38]:
# применим функцию к столбцу доходов
df['income_group'] = df['total_income'].apply(specify_income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,income_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,недвижимость,выше 195818
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль,от 107620 до 142594
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,недвижимость,от 142594 до 195818
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование,выше 195818
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба,от 142594 до 195818


In [39]:
df['income_group'].value_counts() #выведем уникальные значения и их количество

от 107620 до 142594    5479
до 107620              5364
выше 195818            5363
от 142594 до 195818    5247
Name: income_group, dtype: int64

In [40]:
#продублируем категоризацию по возрасту для наглядности
df['age_group'].value_counts()

до 34    5466
43-53    5448
34-43    5284
53+      5255
Name: age_group, dtype: int64

In [41]:
# и категоризацию по цели кредита для наглядности
df['purpose_group'].value_counts()

недвижимость     8932
автомобиль       4306
образование      4013
свадьба          2324
строительство    1878
Name: purpose_group, dtype: int64

**Вывод**


- большим спросом пользуется кредит на недвижимость;
- доход на запрос кредита не влияет;
- кредиту все возрасты покорны

[Наверх к оглавлению](#contents)

<a id="analysis"></a>
## Анализ

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
# построим сводную таблицу применив метод pivot_table
df_pivot = df.pivot_table(index = ['children'], columns = ['debt'], values = 'age_group', aggfunc='count',fill_value=0)

#создадим в таблице df_pivot новый столбец 'ratio' и сохраним в нём значение отношения имеющих задолжность к общему числу в группе
#округлим значения при помощи метода 'round'
df_pivot['ratio'] = round(df_pivot[1] / (df_pivot[0] + df_pivot[1]) * 100, 2)
df_pivot

debt,0,1,ratio
children,,,
0,13027,1063,7.54
1,4410,445,9.17
2,1926,202,9.49
3,303,27,8.18
4,37,4,9.76
5,9,0,0.00


**Вывод**

Вероятность задолженности по выплате кредита при наличии детей увеличивается, чем их отсутствие в семье

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
df_pivot1 = df.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')

#посчитаем вероятность задолженности для каждого вида семейного положения
df_pivot1['ratio'] = round(df_pivot1[1] / (df_pivot1[0] + df_pivot1[1]) * 100, 2)
#применим метод 'sort_values' для наглядности
df_pivot1.sort_values('ratio', ascending = False) 

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,9.75
гражданский брак,3762,388,9.35
женат / замужем,11408,931,7.55
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57


**Вывод**

Вероятность просрочки по кредиту у свободного от семейных уз заёмщика выше, чем у заёмщика в официальном браке, причем и гражданский брак не даёт степень надежности

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
df_pivot2 = df.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
df_pivot2['ratio'] = round(df_pivot2[1] / (df_pivot2[0] + df_pivot2[1]) * 100, 2)
df_pivot2.sort_values('ratio', ascending = False)

debt,0,1,ratio
income_group,,,
от 107620 до 142594,4996,483,8.82
от 142594 до 195818,4799,448,8.54
до 107620,4937,427,7.96
выше 195818,4980,383,7.14



**Вывод**


    
Вероятность задолженности по кредиту у заёмщика с доходом от 107620 рублей до 195818 рублей выше, чем у заёмщика с более высоким  и низким доходом.


- Как разные цели кредита влияют на его возврат в срок?

In [45]:
df_pivot3 = df.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
df_pivot3['ratio'] = round(df_pivot3[1] / (df_pivot3[0] + df_pivot3[1]) * 100, 2)
df_pivot3.sort_values('ratio', ascending = False)

debt,0,1,ratio
purpose_group,,,
автомобиль,3903,403,9.36
образование,3643,370,9.22
свадьба,2138,186,8.00
строительство,1734,144,7.67
недвижимость,8294,638,7.14


**Вывод**

Возможно тем, что недвижимость часто остается в залоге у банка, обусловлен низкий процент вероятности просрочки выплат по кредиту. Тогда как образование и автомобиль менее стабильны и ответственность за выплату по кредиту падает.

[Наверх к оглавлению](#contents)

<a id="general"></a>
## Общий вывод

*В данной проектной работе были заполнены пропущенные значения, удалены дубликаты и выделены категории для целей кредита, группы по возрасту и доходу. Применили сводные таблицы для оценки критериев  возврата кредита. Провели краткий анализ всех вышеперечисленных исследований.*


Важный этап в проверке заемщиков – оценка рисков. Клиенты с доходом ниже 107620  рублей более ответственны, чем остальные клиенты в других группах, за исключением тех, кто имеет доход выше 195818 рублей. Возможно, что это связано с более грамотным распределением своих денежных средств. Просрочки при целевом кредите на недвижимость и строительство ниже, чем на остальные цели. Наличие детей налагает на человека повышенные финансовые обязательства и, по факту, снижает его платежеспособность.  Исходя из изложенного выше, при прочих равных условиях, вероятнее, кредит выдадут человеку женатому, но без детей, нежели человеку с детьми. 